In [ ]:
encoder_path = "./model/encoder/model.onnx"
decoder_path = "./model/decoder/model.onnx"
model_id = "openai/whisper-large-v3-turbo"

ExecutionProvider="QNNExecutionProvider"

In [ ]:
# reference: https://learn.microsoft.com/en-us/windows/ai/new-windows-ml/tutorial?tabs=python#acquiring-the-model-and-preprocessing
import subprocess
import json
import sys
import os
import onnxruntime as ort

def register_execution_providers():
    worker_script = os.path.abspath('winml.py')
    print(worker_script)
    result = subprocess.check_output([sys.executable, worker_script], text=True)
    paths = json.loads(result)
    for item in paths.items():
        try:
            ort.register_execution_provider_library(item[0], item[1])
        except Exception as e:
            print(f"Failed to register execution provider {item[0]}: {e}")

register_execution_providers()

In [ ]:
from urllib import request

test_audio_url = "https://raw.githubusercontent.com/microsoft/onnxruntime-extensions/main/test/data/1272-141231-0002.wav"
test_audio_name = "1272-141231-0002.wav"

request.urlretrieve(test_audio_url, test_audio_name)

In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

import soundfile as sf
from qnn_app import HfWhisperAppWithSave

app = HfWhisperAppWithSave(encoder_path, decoder_path, model_id, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)

audio, sample_rate = sf.read(test_audio_name)
save_data = False

transcription = app.transcribe(audio, sample_rate, test_audio_name, save_data)

from transformers.models.whisper import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_id)
prediction = processor.tokenizer._normalize(transcription)
print("Prediction:", prediction)